In [1]:
import deviantart
from datetime import datetime
import json
from scrape import get_deviations
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def n_stars_diff(url):
    da = deviantart.Api(29316, "4ed6af8f905b519e6e05f82247ee7dee")
    
    all_works = {}
    all_works['title'] = []
    all_works['comments'] = []
    all_works['favourites'] = []
    all_works['published_date'] = []
    all_works['tags'] = []
    all_works['daily_deviation_status'] = []
    
    username = url.split('/')[-1]
    work_count = get_deviations(url)
    offset_count = 0
    
    while work_count > 0:
        works = da.get_gallery_folder(username=username, limit=24, offset=offset_count)
        for work in works['results']:
            tag_data = da.get_deviation_metadata(work.deviationid)[0]['tags']
            tags = []
            for tag in tag_data:
                tags.append(tag['tag_name'])
            all_works['title'].append(work.title)
            all_works['comments'].append(work.stats['comments'])
            all_works['favourites'].append(work.stats['favourites'])
            all_works['published_date'].append(str(datetime.utcfromtimestamp(int(work.published_time))).split(' ')[0])
            all_works['tags'].append(tags)
            all_works['daily_deviation_status'].append((None != work.daily_deviation))
            #all_works[work.title] = [work.stats, str(datetime.utcfromtimestamp(int(work.published_time))), tags, (None != work.daily_deviation)]
        work_count -= 24
        offset_count += 24
        
    df = pd.DataFrame(all_works)
    df = df.sort_values(by='published_date')
    
    n_star = (df['favourites'][0], 0)
    for i in range(1, len(df['favourites'])):
        if df['favourites'][i] > n_star[0]:
            n_star = (df['favourites'][i], i)
            
    n_doublestar = (df['favourites'][0], 0)
    for i in range(1, len(df['favourites'])):
        if df['favourites'][i] > n_doublestar[0] and i != n_star[1]:
            n_doublestar = (df['favourites'][i], i)
            
    return abs(n_star[1] - n_doublestar[1])

In [22]:
differences = []
animal_artists = ["https://www.deviantart.com/thrumyeye", 
"https://www.deviantart.com/innocentium",
 "https://www.deviantart.com/hadissima",
 "https://www.deviantart.com/sergey-ryzhkov",
 "https://www.deviantart.com/thunderi",
 "https://www.deviantart.com/arkus83",
 "https://www.deviantart.com/metalwolf13fh",
 "https://www.deviantart.com/v-k-photography",
 "https://www.deviantart.com/kristynakvapilova",
 "https://www.deviantart.com/glenn0o7",
 "https://www.deviantart.com/quiet-bliss",
 "https://www.deviantart.com/patricksworld",
 "https://www.deviantart.com/zwergloh",
 "https://www.deviantart.com/trablamonron",
 "https://www.deviantart.com/hoofbeatsnpawprints",
 "https://www.deviantart.com/sorance",
 "https://www.deviantart.com/roisabborrar",
 "https://www.deviantart.com/charlyjade",
 "https://www.deviantart.com/hiiakay",
 "https://www.deviantart.com/ducondona",
 "https://www.deviantart.com/monsterbrand",
 "https://www.deviantart.com/azph", 
 "https://www.deviantart.com/huskana"]

In [23]:
for artist in animal_artists:
    differences.append(n_stars_diff(artist))

In [24]:
differences

[464,
 6,
 0,
 27,
 92,
 118,
 8,
 41,
 113,
 0,
 262,
 278,
 146,
 12,
 231,
 22,
 105,
 275,
 64,
 127,
 20,
 282,
 2]

In [25]:

with open("output.txt", "w") as file:
    for item in differences:
        file.write("%s\n" % item)
